# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。





# 比較
* requests vs. grab
    * Grab可用g.setup直接設定 method等各種參數
    * Grab可用.body取出bytes資料，直接用於圖片存取；如需像requests取出str資料(r.text)，則需使用.unicode_body()
* beatifulsoup vs. pyquery
    * pyquery允許對 xml檔 進行jQuery查詢
    * API類似jQuery

## requests + BeautifulSoup


In [2]:
# requests + BeautifulSoup：下載 Yahoo 頭條新聞
import requests
from bs4 import BeautifulSoup

# 下載 Yahoo 首頁內容
r = requests.get('https://tw.yahoo.com/')

# 確認是否下載成功
if r.status_code == requests.codes.ok:
    # 以 BeautifulSoup 解析 HTML 程式碼
    soup = BeautifulSoup(r.text, 'html.parser')
    # 以 CSS 的 class 抓出各類頭條新聞
    stories = soup.find_all('a', class_='story-title')
    a = 0
    for s in stories:
        # 新聞標題
        print("標題：" + s.text)
        # 新聞網址
        print("網址：" + s.get('href'))
        # 只下載前三則
        a += 1
        if a == 3:
            break

標題：暗中追查一年 女檢座逮到卡神
網址：https://tw.news.yahoo.com/%E5%8D%A1%E7%A5%9E%E5%90%A6%E8%AA%8D%E4%B9%9F%E4%B8%8D%E6%94%BE%E6%A3%84%E8%BF%BD%E6%9F%A5-%E8%8F%AF%E5%B1%B1%E5%88%86%E5%B1%8D%E6%A1%88%E5%A5%B3%E6%AA%A2%E8%8A%B1%E5%B9%B4%E9%A4%98%E9%80%AE%E5%88%B0%E5%A5%B9-024224521.html
標題：踏上政治路 宣明智公開信吐心聲
網址：https://tw.news.yahoo.com/%E5%AE%A3%E6%98%8E%E6%99%BA%E7%B5%A6%E5%A5%BD%E5%8F%8B-%E5%B0%81%E4%BF%A1-%E8%AE%93%E5%B0%8D%E7%9A%84%E4%BA%BA%E8%A6%81%E6%B1%82%E6%94%BF%E5%BA%9C%E5%81%9A%E5%B0%8D%E7%9A%84%E4%BA%8B-203138762.html
標題：人夫愛上人妻 攜手5度進旅館
網址：https://tw.news.yahoo.com/%E5%80%8B%E6%9C%88%E5%95%AA5%E6%AC%A1-%E4%BA%BA%E5%A4%AB%E6%BF%80%E6%88%B0%E4%BA%BA%E5%A6%BB%E6%85%98%E5%99%B436%E8%90%AC-050900984.html


## Grab + PyQuery

In [85]:
# Grab + PyQuery：下載 Yahoo 頭條新聞
from grab import Grab
from pyquery import PyQuery as pq

g = Grab()
url = 'https://tw.yahoo.com/'
resp = g.go(url)
print(type(resp.body))
print(type(resp.unicode_body()))

doc = pq(resp.unicode_body())
a = 0
for i in range(len(doc('a')('.story-title'))):
    print('標題：', doc('a')('.story-title').eq(i).text())
    print('網址：', doc('a')('.story-title').eq(i).attr('href'))
    # 只下載前三則
    a += 1
    if a == 3:
        break

<class 'bytes'>
<class 'str'>
標題： 張善政老神在在：韓藏有殺手鐧
網址： https://tw.news.yahoo.com/%E6%B0%91%E8%AA%BF%E8%90%BD%E5%BE%8C%E6%B2%92%E5%9C%A8%E6%80%95-%E5%BC%B5%E5%96%84%E6%94%BF%E6%9B%9D-%E9%9F%93%E9%99%A3%E7%87%9F%E8%97%8F%E6%9C%89-%E6%AE%BA%E6%89%8B%E9%90%A7-233315981.html
標題： 觀看數達431萬 最熱門影片出爐
網址： https://tw.news.yahoo.com/youtube%E5%B9%B4%E5%BA%A6%E6%9C%80%E5%8F%97%E6%AD%A1%E8%BF%8E%E5%BD%B1%E7%89%87-%E5%8F%88%E4%BE%86%E6%8C%91%E6%88%B0%E9%A4%A8%E9%95%B7-082924445.html
標題： 「氣爆案用騙的」陳菊告輸了
網址： https://tw.news.yahoo.com/%E9%99%B3%E8%8F%8A%E8%BC%B8%E4%BA%86%E4%B8%8D%E6%BB%BF%E8%AC%9D%E5%AF%92%E5%86%B0%E6%8C%87%E9%AB%98%E9%9B%84%E6%B0%A3%E7%88%86%E6%A1%88%E7%94%A8%E9%A8%99%E7%9A%84%E6%80%92%E5%91%8A%E8%AA%B9%E8%AC%97%E4%B8%80%E5%AF%A9%E5%88%A4%E7%84%A1%E7%BD%AA-084748850.html
